In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from datetime import date as dt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

from collections import Counter
import warnings
warnings.filterwarnings('ignore')

In [3]:
df=pd.read_csv('companies.csv')


In [4]:
df.head()

,id,Unnamed: 0.1,entity_type,entity_id,parent_id,name,normalized_name,permalink,category_code,status,...,first_milestone_at,last_milestone_at,milestones,relationships,created_by,created_at,updated_at,lat,lng,ROI
0,c:1,0,Company,1,NaN,Wetpaint,wetpaint,/company/wetpaint,web,operating,...,2010-09-05,2013-09-18,5.0,17.0,initial-importer,2007-05-25 06:51:27,2013-04-13 03:29:00,47.606209,-122.332071,15.5
1,c:10,1,Company,10,NaN,Flektor,flektor,/company/flektor,games_video,acquired,...,NaN,NaN,NaN,6.0,initial-importer,2007-05-31 21:11:51,2008-05-23 23:23:14,34.021122,-118.396467,NaN
2,c:100,2,Company,100,NaN,There,there,/company/there,games_video,acquired,...,2003-02-01,2011-09-23,4.0,12.0,initial-importer,2007-08-06 23:52:45,2013-11-04 02:09:48,37.562992,-122.325525,NaN
3,c:10000,3,Company,10000,NaN,MYWEBBO,mywebbo,/company/mywebbo,network_hosting,operating,...,NaN,NaN,NaN,NaN,NaN,2008-08-24 16:51:57,2008-09-06 14:19:18,NaN,NaN,NaN
4,c:10001,4,Company,10001,NaN,THE Movie Streamer,the movie streamer,/company/the-movie-streamer,games_video,operating,...,NaN,NaN,NaN,NaN,NaN,2008-08-24 17:10:34,2008-09-06 14:19:18,NaN,NaN,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196553 entries, 0 to 196552
Data columns (total 44 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   196553 non-null  object 
 1   Unnamed: 0.1         196553 non-null  int64  
 2   entity_type          196553 non-null  object 
 3   entity_id            196553 non-null  int64  
 4   parent_id            0 non-null       float64
 5   name                 196531 non-null  object 
 6   normalized_name      196527 non-null  object 
 7   permalink            196553 non-null  object 
 8   category_code        123186 non-null  object 
 9   status               196553 non-null  object 
 10  founded_at           91227 non-null   object 
 11  closed_at            2620 non-null    object 
 12  domain               126545 non-null  object 
 13  homepage_url         126545 non-null  object 
 14  twitter_username     80591 non-null   object 
 15  logo_url         

In [6]:
df.shape

(196553, 44)

In [7]:
df.duplicated().sum()


0

In [8]:
df.isna().sum()

id                          0
Unnamed: 0.1                0
entity_type                 0
entity_id                   0
parent_id              196553
name                       22
normalized_name            26
permalink                   0
category_code           73367
status                      0
founded_at             105326
closed_at              193933
domain                  70008
homepage_url            70008
twitter_username       115962
logo_url                86443
logo_width              86443
logo_height             86443
short_description      189422
description            104505
overview                69582
tag_list               115101
country_code           108563
state_code             145650
city                   112663
region                      0
first_investment_at    193970
last_investment_at     193970
investment_rounds      193962
invested_companies     193962
first_funding_at       165046
last_funding_at        165046
funding_rounds         164846
funding_to

In [9]:
data = df.drop(columns = ['id','entity_type','permalink','region','created_by','Unnamed: 0.1','normalized_name','parent_id','domain','homepage_url','logo_url','logo_width','logo_height','overview','short_description','twitter_username','short_description','description','tag_list','state_code','city','first_investment_at', 'last_investment_at','first_milestone_at','last_milestone_at',  'first_funding_at','last_funding_at','lat','lng'])
data.head()

,entity_id,name,category_code,status,founded_at,closed_at,country_code,investment_rounds,invested_companies,funding_rounds,funding_total_usd,milestones,relationships,created_at,updated_at,ROI
0,1,Wetpaint,web,operating,2005-10-17,NaN,USA,NaN,NaN,3.0,39750000.0,5.0,17.0,2007-05-25 06:51:27,2013-04-13 03:29:00,15.5
1,10,Flektor,games_video,acquired,NaN,NaN,USA,NaN,NaN,NaN,NaN,NaN,6.0,2007-05-31 21:11:51,2008-05-23 23:23:14,NaN
2,100,There,games_video,acquired,NaN,NaN,USA,NaN,NaN,NaN,NaN,4.0,12.0,2007-08-06 23:52:45,2013-11-04 02:09:48,NaN
3,10000,MYWEBBO,network_hosting,operating,2008-07-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008-08-24 16:51:57,2008-09-06 14:19:18,NaN
4,10001,THE Movie Streamer,games_video,operating,2008-07-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008-08-24 17:10:34,2008-09-06 14:19:18,NaN


In [10]:
data.shape

(196553, 16)

In [11]:
data.dropna(subset= ['name','country_code','category_code','founded_at'],inplace = True)
data.shape

(64094, 16)

In [12]:
data['investment_rounds'] = data['investment_rounds'].fillna(data['investment_rounds'].median())


In [13]:
data['invested_companies'].value_counts()


1.0      460
2.0      104
3.0       56
4.0       26
5.0       23
6.0       14
7.0       13
14.0       6
16.0       5
9.0        4
8.0        4
11.0       2
10.0       2
29.0       2
38.0       2
26.0       2
18.0       2
24.0       2
21.0       2
20.0       2
57.0       1
68.0       1
44.0       1
124.0      1
12.0       1
39.0       1
79.0       1
268.0      1
30.0       1
73.0       1
459.0      1
33.0       1
13.0       1
47.0       1
17.0       1
51.0       1
Name: invested_companies, dtype: int64

In [14]:
data['invested_companies'] = data['invested_companies'].fillna(data['invested_companies'].median())


In [15]:
data['funding_rounds'].value_counts()


1.0     13378
2.0      4964
3.0      2367
4.0      1100
5.0       561
6.0       239
7.0       136
8.0        62
9.0        42
10.0       22
11.0       10
13.0        4
12.0        3
15.0        2
14.0        1
Name: funding_rounds, dtype: int64

In [17]:
data['funding_rounds'] = data['funding_rounds'].fillna(data['funding_rounds'].median())


In [18]:
data['funding_total_usd'] = data['funding_total_usd'].fillna(data['funding_total_usd'].mean())


In [19]:
data['milestones'].value_counts()

1.0    24691
2.0     7478
3.0     2478
4.0      458
5.0      139
6.0       45
7.0        7
8.0        3
9.0        1
Name: milestones, dtype: int64

In [20]:
data['milestones'] = data['milestones'].fillna(data['milestones'].mode()[0])


In [21]:
data['relationships'].value_counts()


1.0      13943
2.0      10982
3.0       7221
4.0       4384
5.0       2723
         ...  
183.0        1
155.0        1
122.0        1
387.0        1
67.0         1
Name: relationships, Length: 137, dtype: int64

In [22]:
data['relationships'] = data['relationships'].fillna(data['relationships'].mean())


In [23]:
data['ROI'].value_counts()


10.000000    6
15.000000    4
16.666667    3
3.333333     3
5.000000     3
            ..
57.777778    1
0.232143     1
7.788945     1
0.675219     1
24.137931    1
Name: ROI, Length: 504, dtype: int64

In [25]:
data['ROI'] = data['ROI'].fillna(data['ROI'].mean())


In [26]:
data['founded_at'] = pd.to_datetime(data['founded_at'])
data['founded_at'] = data['founded_at'].dt.year


In [27]:
data['founded_at'] = data['founded_at'].fillna(data['founded_at'].mode()[0])


In [28]:
data['closed_at'] = pd.to_datetime(data['closed_at'])
data['closed_at'] = data['closed_at'].dt.year

In [29]:
data['closed_at'].mode()[0]


2012.0

In [30]:
for i in data['status']:
    if(i == 'operating' or 'ipo'):
        data['closed_at'].fillna(2021, inplace = True)
    elif (i == 'acquired' or 'closed'):
        data['closed_at'].fillna(2012, inplace = True)


In [31]:
created = []
for i in data['created_at']:
    created.append(i.split('-')[0])
data['created_at'] = created

In [32]:
created = []
for i in data['updated_at']:
    created.append(int(i.split('-')[0]))
data['updated_at']= created


In [33]:
data['companies_age'] = data['closed_at'] - data['founded_at']


In [36]:
index = np.where(data['companies_age'] < 0)
data['companies_age'] = data['companies_age'].drop(data['companies_age'].index[index])


In [37]:
data['companies_age'] = data['companies_age'].fillna(0)


In [38]:
data.head()

,entity_id,name,category_code,status,founded_at,closed_at,country_code,investment_rounds,invested_companies,funding_rounds,funding_total_usd,milestones,relationships,created_at,updated_at,ROI,companies_age
0,1,Wetpaint,web,operating,2005,2021.0,USA,1.0,1.0,3.0,3.975000e+07,5.0,17.00000,2007,2013,15.50000,16.0
5,10002,Synergie Media,advertising,operating,2007,2021.0,MAR,1.0,1.0,1.0,1.581978e+07,1.0,2.00000,2008,2008,14.41045,14.0
6,10003,Green Basti Ecosystems,cleantech,operating,2008,2021.0,IND,1.0,1.0,1.0,1.581978e+07,1.0,4.43465,2008,2010,14.41045,13.0
12,10009,dimension5 labs,advertising,operating,2008,2021.0,USA,1.0,1.0,1.0,1.581978e+07,1.0,2.00000,2008,2008,14.41045,13.0
13,1001,FriendFeed,web,acquired,2007,2021.0,USA,1.0,1.0,1.0,5.000000e+06,3.0,14.00000,2007,2013,9.50000,14.0


In [39]:
data.isna().sum()


entity_id             0
name                  0
category_code         0
status                0
founded_at            0
closed_at             0
country_code          0
investment_rounds     0
invested_companies    0
funding_rounds        0
funding_total_usd     0
milestones            0
relationships         0
created_at            0
updated_at            0
ROI                   0
companies_age         0
dtype: int64

In [40]:
data.reset_index(inplace = True)


In [41]:
data.shape


(64094, 18)

In [42]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
data['encoded_column'] = label_encoder.fit_transform(data['status'])

In [43]:
from sklearn.preprocessing import LabelEncoder

# Assuming 'df' is your DataFrame
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data['status'])

# Display the mapping between original values and encoded values
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label Mapping:", label_mapping)


Label Mapping: {'acquired': 0, 'closed': 1, 'ipo': 2, 'operating': 3}


In [45]:
label_mapping = {'acquired': 0, 'closed': 1, 'ipo': 2, 'operating': 3}
df['label'] = df['status'].map(label_mapping)
df['label']

0         3
1         0
2         0
3         3
4         3
         ..
196548    3
196549    3
196550    3
196551    3
196552    3
Name: label, Length: 196553, dtype: int64

In [46]:
aggregated_group = data.groupby('status')[['milestones', 'relationships']].mean()
print(aggregated_group)

           milestones  relationships
status                              
acquired     1.589231       6.746131
closed       1.324739       3.563485
ipo          2.135621      33.906004
operating    1.192190       3.992358


In [47]:
pivot_table = data.pivot_table(index='status', columns='category_code', values='investment_rounds', aggfunc='mean')
print(pivot_table)

category_code  advertising  analytics  automotive   biotech  cleantech  \
status                                                                   
acquired          1.008929        1.0         1.0  1.008163   1.000000   
closed            1.000000        1.0         1.0  1.000000   1.000000   
ipo               1.150000        1.0         1.0  1.550000   1.259259   
operating         1.002306        1.0         1.0  1.010309   1.007964   

category_code  consulting  design  ecommerce  education  enterprise  ...  \
status                                                               ...   
acquired         1.000000     1.0   1.018519   1.000000    1.008163  ...   
closed           1.000000     1.0   1.000000   1.000000    1.000000  ...   
ipo              1.166667     NaN   2.421053   1.000000    2.781250  ...   
operating        1.024781     1.0   1.001682   1.003416    1.015536  ...   

category_code  real_estate    search  security  semiconductor    social  \
status                 

In [49]:
avg_software_for_ipo = pivot_table.loc['ipo', 'software']
print("Average value of software for IPO companies:", avg_software_for_ipo)


Average value of software for IPO companies: 1.8666666666666667


In [51]:
lowest_avg_category_acquired = pivot_table.loc['acquired'].idxmin()
print("Category with average value for acquired companies:", lowest_avg_category_acquired)


Category with average value for acquired companies: analytics


In [53]:
total_avg_value_operating = pivot_table.loc['operating'].mean()
print("Total average value of all categories for operating companies:", total_avg_value_operating)


Total average value of all categories for operating companies: 1.0550325996182535


In [55]:
total_companies_software = pivot_table['software'].sum()
print("Total number of companies in the 'software' category:", total_companies_software)


Total number of companies in the 'software' category: 4.886160265605406
